In [1]:
"""
Queries each sudo pool for pair specific attributes
to build local csv of all pair pools.
"""
# since pools are not verified on Etherscan, need to use local contracts
# switch to sudo-ape wd first for ape project to work properly
import os
import time
from pathlib import Path

if Path(*Path(os.getcwd()).parts[-3:]) == Path('sudo-ape/notebook/queries'):
    os.chdir('../..')

In [3]:
import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks, project
from math import ceil

In [4]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__() 

<alchemy chain_id=1>

In [5]:
# check LSSVMPair in local project
project.LSSVMPair

<LSSVMPair>

In [6]:
# load sudo pairs from csv output of ./sudo-pairs.ipynb
query_pairs = pd.read_csv('notebook/data/pairs_raw.csv')

In [7]:
query_pairs

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x08142348E6BBf233002b81047bc...,0xba75dd0b017667074a1ed080571fee06b3cea5444f25...,14650748,0xf63250e045af2d83fca5535fe3738deffd3c6f17f93b...,15,9,0x08142348E6BBf233002b81047bc2f27026af10A5
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x518b7160990AD19dcfF0Ba028e2...,0x7e33fe2f08ee09a339b127c8658a892600637da3448d...,14652189,0x4b0de468eecdfea07b64168655f129d43d8e56f01f42...,41,37,0x518b7160990AD19dcfF0Ba028e245fA616b44F5E
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x351E40e39ae7D0d60Dc283485bC...,0x230624db7d760f3ad189bdebb431ef74b6cd899855d6...,14652199,0x572e74e542e87e141a2176725d7b216133bd0d13dcf5...,150,117,0x351E40e39ae7D0d60Dc283485bC627891Ec1D00E
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x0636a6Bd07dDBEA28a3cc63353e...,0xff03f1a4956f878f1bd42a6dfa2ec18165822cde3d15...,14652223,0x877d4993fe635aaaea71fa3c8bb8c705d67f86559c3e...,142,108,0x0636a6Bd07dDBEA28a3cc63353ea725e8cE509e2
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x866C5c02dC07EE37146c4874d70...,0x0c2b148a424bb8f25f32ca3ded2c87704cb73ab5b65b...,14662427,0x8f687e0ea159efeb37fd7e65ec00a008e6b96cc1b7ee...,171,149,0x866C5c02dC07EE37146c4874d704D3603bF5D950
...,...,...,...,...,...,...,...,...,...
30990,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9AAa20179145C9153a36Bbf71e2...,0x35feb962e106bfbd16bb949605d171bda649c015621d...,15561562,0x7fe005a4046800acc48f09ae4a0c92eea79f0cf0a38f...,481,265,0x9AAa20179145C9153a36Bbf71e2FF0c6eC2bed1f
30991,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xf29EE6A9C566Fb837988a72bdaa...,0xf8451b200c08f1603db95f39140fb21d633f6119e9f4...,15561651,0x3c1e8ab50f8a8dcba3e5fae236eb5ec6640bf942d0e3...,146,92,0xf29EE6A9C566Fb837988a72bdaa5749564Dc8b69
30992,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xCbb272D3c518386D1dd5a673698...,0x5eb463db486d91a5ee20d64f894d504b03820bcde553...,15561725,0x03a607aa73430521c37d30a177c1de30509b46a143fd...,110,39,0xCbb272D3c518386D1dd5a673698F02d1a0063D3c
30993,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xC66Cfab72F2d43eb6Bb7058B2af...,0xe482845e9b8234d98b0e6ed1747b62bafc6ebc345dad...,15561734,0x6889adf6bf42faf501840a41858d7687860edc3e1138...,81,72,0xC66Cfab72F2d43eb6Bb7058B2af484A45eB2b9E9


In [8]:
# filter out non-TRADE type pools to make querying easier, since we only care
# about collections with some trade liquidity.
# poolType == 2 is TRADE
sudo_pair_xmons = Contract("0x5caf332dca4e6c9e69d52f320c21e74845353db0")
sudo_pair_xmons.poolType()

2

In [9]:
# NewPair events only give the pair address without any addition. Query
# for additional pool info from each pool contract at poolAddress in NewPair
def get_pair_specifics(key: str, x: pd.Series, is_method: bool = True) -> tp.Any:
    """
    Gets pair specific info from the LSSVMPair contract.
    """
    addr = x['poolAddress']
    pair = project.LSSVMPair.at(addr)
    try:
        val = getattr(pair, key)() if is_method else getattr(pair, key)
    except Exception:
        val = None
    return val

In [10]:
# start out with poolType & nft info to join into query_pairs df
# chunk requests since otherwise issues with alchemy
chunk_size = 4000
num_chunks = ceil(len(query_pairs) / chunk_size)

In [11]:
# read in poolType chunks we already fetched and stored in csv (so don't have to query as many to catchup)
query_pool_types = pd.read_csv('notebook/data/specifics/pool_types.csv')

In [12]:
query_pool_types

,poolType
0,1
1,1
2,1
3,1
4,1
...,...
30990,1
30991,2
30992,1
30993,1


In [ ]:
# determine the chunk id currently on given csv file load ...
idx_chunk = int(len(query_pool_types) / chunk_size)

In [ ]:
# repeat execution of this until get specifics on poolType to filter out poolType!=2 pools
if idx_chunk < num_chunks+1:
    print('idx_chunk (before)', idx_chunk)
    %time query_pairs_pool_chunk = query_pairs[chunk_size*idx_chunk:chunk_size*(idx_chunk+1)].apply(lambda x: get_pair_specifics('poolType', x), axis=1)
    print('query_pairs_pool_chunk', query_pairs_pool_chunk)
    query_pool_types = pd.concat([query_pool_types, pd.DataFrame(data={'poolType': query_pairs_pool_chunk})])
    idx_chunk += 1
    print('idx_chunk (after)', idx_chunk)

In [13]:
# reduces number of pools by factor of ~ 10
query_pool_types[query_pool_types.poolType == 2]

,poolType
6,2
8,2
10,2
11,2
12,2
...,...
30984,2
30985,2
30986,2
30988,2


In [15]:
# save temp to csv for easy loading above in case mess up in nb
query_pool_types.to_csv('notebook/data/specifics/pool_types.csv', index=False)

In [14]:
# add pool type column to query pairs
query_pairs['poolType'] = query_pool_types

In [15]:
query_pairs

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress,poolType
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x08142348E6BBf233002b81047bc...,0xba75dd0b017667074a1ed080571fee06b3cea5444f25...,14650748,0xf63250e045af2d83fca5535fe3738deffd3c6f17f93b...,15,9,0x08142348E6BBf233002b81047bc2f27026af10A5,1
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x518b7160990AD19dcfF0Ba028e2...,0x7e33fe2f08ee09a339b127c8658a892600637da3448d...,14652189,0x4b0de468eecdfea07b64168655f129d43d8e56f01f42...,41,37,0x518b7160990AD19dcfF0Ba028e245fA616b44F5E,1
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x351E40e39ae7D0d60Dc283485bC...,0x230624db7d760f3ad189bdebb431ef74b6cd899855d6...,14652199,0x572e74e542e87e141a2176725d7b216133bd0d13dcf5...,150,117,0x351E40e39ae7D0d60Dc283485bC627891Ec1D00E,1
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x0636a6Bd07dDBEA28a3cc63353e...,0xff03f1a4956f878f1bd42a6dfa2ec18165822cde3d15...,14652223,0x877d4993fe635aaaea71fa3c8bb8c705d67f86559c3e...,142,108,0x0636a6Bd07dDBEA28a3cc63353ea725e8cE509e2,1
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x866C5c02dC07EE37146c4874d70...,0x0c2b148a424bb8f25f32ca3ded2c87704cb73ab5b65b...,14662427,0x8f687e0ea159efeb37fd7e65ec00a008e6b96cc1b7ee...,171,149,0x866C5c02dC07EE37146c4874d704D3603bF5D950,1
...,...,...,...,...,...,...,...,...,...,...
30990,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9AAa20179145C9153a36Bbf71e2...,0x35feb962e106bfbd16bb949605d171bda649c015621d...,15561562,0x7fe005a4046800acc48f09ae4a0c92eea79f0cf0a38f...,481,265,0x9AAa20179145C9153a36Bbf71e2FF0c6eC2bed1f,1
30991,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xf29EE6A9C566Fb837988a72bdaa...,0xf8451b200c08f1603db95f39140fb21d633f6119e9f4...,15561651,0x3c1e8ab50f8a8dcba3e5fae236eb5ec6640bf942d0e3...,146,92,0xf29EE6A9C566Fb837988a72bdaa5749564Dc8b69,2
30992,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xCbb272D3c518386D1dd5a673698...,0x5eb463db486d91a5ee20d64f894d504b03820bcde553...,15561725,0x03a607aa73430521c37d30a177c1de30509b46a143fd...,110,39,0xCbb272D3c518386D1dd5a673698F02d1a0063D3c,1
30993,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xC66Cfab72F2d43eb6Bb7058B2af...,0xe482845e9b8234d98b0e6ed1747b62bafc6ebc345dad...,15561734,0x6889adf6bf42faf501840a41858d7687860edc3e1138...,81,72,0xC66Cfab72F2d43eb6Bb7058B2af484A45eB2b9E9,1


In [16]:
# read in nft collection chunks we already fetched and stored in csv (so don't have to query as many to catchup)
query_nfts = pd.read_csv('notebook/data/specifics/nfts.csv')

In [17]:
# determine the chunk id currently on given csv file load ...
idx_chunk = int(len(query_nfts) / chunk_size)

In [23]:
# repeat execution of this until get specifics on nft to have collections for all pools
if idx_chunk < num_chunks+1:
    print('idx_chunk (before)', idx_chunk)
    %time query_pairs_nft_chunk = query_pairs[chunk_size*idx_chunk:chunk_size*(idx_chunk+1)].apply(lambda x: get_pair_specifics('nft', x), axis=1)
    print('query_pairs_nft_chunk', query_pairs_nft_chunk)
    query_nfts = pd.concat([query_nfts, pd.DataFrame(data={'nft': query_pairs_nft_chunk})])
    idx_chunk += 1
    print('idx_chunk (after)', idx_chunk)

idx_chunk (before) 7
CPU times: user 14min 5s, sys: 2min 55s, total: 17min 1s
Wall time: 39min 9s
query_pairs_nft_chunk 28000    0x8D94B8Ea78A51D783A413c73811511c900C8F365
28001    0x837704Ec8DFEC198789baF061D6e93B0e1555dA6
28002    0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85
28003    0xCa21d4228cDCc68D4e23807E5e370C07577Dd152
28004    0x65784d6F23DE30A17122E96c0F0986C378ed6947
                            ...                    
30990    0x1CEf785AA885836CeE4f53B6ACe83D187d819639
30991    0x8270FC3B2d23DE703b265b2ABE008883954fea8E
30992    0x1CEf785AA885836CeE4f53B6ACe83D187d819639
30993    0x1CEf785AA885836CeE4f53B6ACe83D187d819639
30994    0x8270FC3B2d23DE703b265b2ABE008883954fea8E
Length: 2995, dtype: object
idx_chunk (after) 8


In [24]:
idx_chunk

8

In [25]:
num_chunks

8

In [27]:
query_nfts

,nft
0,0x5Af0D9827E0c53E4799BB226655A1de152A425a5
1,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD
2,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42
3,0x0427743DF720801825a5c82e0582B1E915E0F750
4,0xD07e72b00431af84AD438CA995Fd9a7F0207542d
...,...
30990,0x1CEf785AA885836CeE4f53B6ACe83D187d819639
30991,0x8270FC3B2d23DE703b265b2ABE008883954fea8E
30992,0x1CEf785AA885836CeE4f53B6ACe83D187d819639
30993,0x1CEf785AA885836CeE4f53B6ACe83D187d819639


In [26]:
# save temp to csv for easy loading above in case mess up in nb
query_nfts.to_csv('notebook/data/specifics/nfts.csv', index=False)

In [28]:
# add nft column to query pairs
query_pairs['nft'] = query_nfts

In [29]:
query_pairs

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress,poolType,nft
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x08142348E6BBf233002b81047bc...,0xba75dd0b017667074a1ed080571fee06b3cea5444f25...,14650748,0xf63250e045af2d83fca5535fe3738deffd3c6f17f93b...,15,9,0x08142348E6BBf233002b81047bc2f27026af10A5,1,0x5Af0D9827E0c53E4799BB226655A1de152A425a5
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x518b7160990AD19dcfF0Ba028e2...,0x7e33fe2f08ee09a339b127c8658a892600637da3448d...,14652189,0x4b0de468eecdfea07b64168655f129d43d8e56f01f42...,41,37,0x518b7160990AD19dcfF0Ba028e245fA616b44F5E,1,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x351E40e39ae7D0d60Dc283485bC...,0x230624db7d760f3ad189bdebb431ef74b6cd899855d6...,14652199,0x572e74e542e87e141a2176725d7b216133bd0d13dcf5...,150,117,0x351E40e39ae7D0d60Dc283485bC627891Ec1D00E,1,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x0636a6Bd07dDBEA28a3cc63353e...,0xff03f1a4956f878f1bd42a6dfa2ec18165822cde3d15...,14652223,0x877d4993fe635aaaea71fa3c8bb8c705d67f86559c3e...,142,108,0x0636a6Bd07dDBEA28a3cc63353ea725e8cE509e2,1,0x0427743DF720801825a5c82e0582B1E915E0F750
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x866C5c02dC07EE37146c4874d70...,0x0c2b148a424bb8f25f32ca3ded2c87704cb73ab5b65b...,14662427,0x8f687e0ea159efeb37fd7e65ec00a008e6b96cc1b7ee...,171,149,0x866C5c02dC07EE37146c4874d704D3603bF5D950,1,0xD07e72b00431af84AD438CA995Fd9a7F0207542d
...,...,...,...,...,...,...,...,...,...,...,...
30990,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9AAa20179145C9153a36Bbf71e2...,0x35feb962e106bfbd16bb949605d171bda649c015621d...,15561562,0x7fe005a4046800acc48f09ae4a0c92eea79f0cf0a38f...,481,265,0x9AAa20179145C9153a36Bbf71e2FF0c6eC2bed1f,1,0x1CEf785AA885836CeE4f53B6ACe83D187d819639
30991,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xf29EE6A9C566Fb837988a72bdaa...,0xf8451b200c08f1603db95f39140fb21d633f6119e9f4...,15561651,0x3c1e8ab50f8a8dcba3e5fae236eb5ec6640bf942d0e3...,146,92,0xf29EE6A9C566Fb837988a72bdaa5749564Dc8b69,2,0x8270FC3B2d23DE703b265b2ABE008883954fea8E
30992,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xCbb272D3c518386D1dd5a673698...,0x5eb463db486d91a5ee20d64f894d504b03820bcde553...,15561725,0x03a607aa73430521c37d30a177c1de30509b46a143fd...,110,39,0xCbb272D3c518386D1dd5a673698F02d1a0063D3c,1,0x1CEf785AA885836CeE4f53B6ACe83D187d819639
30993,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xC66Cfab72F2d43eb6Bb7058B2af...,0xe482845e9b8234d98b0e6ed1747b62bafc6ebc345dad...,15561734,0x6889adf6bf42faf501840a41858d7687860edc3e1138...,81,72,0xC66Cfab72F2d43eb6Bb7058B2af484A45eB2b9E9,1,0x1CEf785AA885836CeE4f53B6ACe83D187d819639


In [30]:
# save full pairs df (with nft collection and poolType) to csv
query_pairs.to_csv('notebook/data/pairs_full.csv', index=False)

In [31]:
# filter originally loaded pairs by poolType == 2 indices
query_pairs_trade = query_pairs.copy()[query_pairs.poolType == 2]

In [32]:
# get the bonding curve addresses for each TRADE pool
%time query_bonding_curves = query_pairs_trade.apply(lambda x: get_pair_specifics('bondingCurve', x), axis=1)
query_bonding_curves

CPU times: user 19min 19s, sys: 3min 53s, total: 23min 13s
Wall time: 54min 48s


6        0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee
8        0x432f962D8209781da23fB37b6B59ee15dE7d9841
10       0x432f962D8209781da23fB37b6B59ee15dE7d9841
11       0x432f962D8209781da23fB37b6B59ee15dE7d9841
12       0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee
                            ...                    
30984    0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee
30985    0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee
30986    0x432f962D8209781da23fB37b6B59ee15dE7d9841
30988    0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee
30991    0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee
Length: 4109, dtype: object

In [33]:
# save temp to csv for easy loading above in case mess up in nb
query_bonding_curves.to_csv('notebook/data/specifics/trade/bonding_curves.csv', index=False)

In [35]:
# add bonding curves to filtered pairs
query_pairs_trade['bondingCurve'] = query_bonding_curves

In [36]:
# get the fee for each TRADE pool
%time query_fee = query_pairs_trade.apply(lambda x: get_pair_specifics('fee', x), axis=1)
query_fee

CPU times: user 19min 14s, sys: 4min 9s, total: 23min 24s
Wall time: 54min 25s


6         19000000000000000
8        150000000000000000
10                        0
11       200000000000000000
12        20000000000000000
                ...        
30984     50000000000000000
30985     50000000000000000
30986     43000000000000000
30988     20000000000000000
30991     50000000000000000
Length: 4109, dtype: int64

In [37]:
# save temp to csv for easy loading above in case mess up in nb
query_fee.to_csv('notebook/data/specifics/trade/fees.csv', index=False)

In [38]:
query_pairs_trade['fee'] = query_fee

In [39]:
# get the delta for each TRADE pool
%time query_delta = query_pairs_trade.apply(lambda x: get_pair_specifics('delta', x), axis=1)
query_delta

CPU times: user 19min 23s, sys: 3min 56s, total: 23min 19s
Wall time: 55min 21s


6          50000000000000000
8        1015000000000000000
10       1030000000000000000
11       1050000000000000000
12         10000000000000000
                ...         
30984       3000000000000000
30985       3000000000000000
30986    1024000000000000000
30988     280000000000000000
30991      50000000000000000
Length: 4109, dtype: uint64

In [40]:
# save temp to csv for easy loading above in case mess up in nb
query_delta.to_csv('notebook/data/specifics/trade/deltas.csv', index=False)

In [41]:
query_pairs_trade['delta'] = query_delta

In [42]:
query_pairs_trade

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress,poolType,nft,bondingCurve,fee,delta
6,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xb149B7754197A99a62BCE1Fe3b0...,0x2294003a296ff7cfca23608e3e0fac22af994a95c021...,14663650,0xe12f8b2c8a8ddfe79c3453dbf7286d6ab300c346d582...,200,219,0xb149B7754197A99a62BCE1Fe3b06309cb40fE248,2,0x0c2E57EFddbA8c768147D1fdF9176a0A6EBd5d83,0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee,19000000000000000,50000000000000000
8,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x628eE9e454ead43F38C2Af03726...,0x1b3b4183b14c2d03ec6f47d5f92e927b02ba43ec03cb...,14664472,0x5a776cdefc76cb68cf62a5679491ad15ba4c412e9072...,268,168,0x628eE9e454ead43F38C2Af03726D295A2f0d17d0,2,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,0x432f962D8209781da23fB37b6B59ee15dE7d9841,150000000000000000,1015000000000000000
10,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x4a0d159356a1Fb6C5A7beA8871E...,0xf18c4577b374e1eecf7bd02661b61835f803026ffa9e...,14719026,0xe628c3b0dd88c24b65107bbcc827ba478f75a78632bc...,240,184,0x4a0d159356a1Fb6C5A7beA8871EeEE8E65c87751,2,0x5Af0D9827E0c53E4799BB226655A1de152A425a5,0x432f962D8209781da23fB37b6B59ee15dE7d9841,0,1030000000000000000
11,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x68325a925f4dE169B083d440135...,0x83efe653e6a05f9317b2ba690b2b59af376820a8eeb0...,14721382,0x35f9f56abf8fc0549e67115223790833de1bb07d70ff...,193,109,0x68325a925f4dE169B083d4401353cEa59B1DEe23,2,0xCa21d4228cDCc68D4e23807E5e370C07577Dd152,0x432f962D8209781da23fB37b6B59ee15dE7d9841,200000000000000000,1050000000000000000
12,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x910bD33634389EAD54D2D34b206...,0xed9b905caa6535a961a2303ea6c6eeafa7df468b8161...,14721400,0x73de95366179611c6dee4a3708af5b0c9e50fe684c7a...,95,163,0x910bD33634389EAD54D2D34b2063c0F452F762a3,2,0xCa21d4228cDCc68D4e23807E5e370C07577Dd152,0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee,20000000000000000,10000000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30984,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x12eFA7A08E9aE088f58BDb65ffe...,0xfe46338b7c1db3604054b6beec87ad3afa698c759616...,15561460,0x8b9ed51024d76188c3a77f26f687194c59c8f377f2f6...,79,34,0x12eFA7A08E9aE088f58BDb65ffe7b5a24E520697,2,0xB39f5F5199042086D6F162EdEDb761D0F33a2849,0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee,50000000000000000,3000000000000000
30985,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xcB492BF99c35aa52D810ccf10a1...,0x8eda440cdcc04ff0d791c98ba6f2a62ddd037aeaf17c...,15561468,0x081f464ed5a0dc3398c0afd1e06d8c0b25a65b98256b...,14,7,0xcB492BF99c35aa52D810ccf10a1a5deBfdCdbd76,2,0xB39f5F5199042086D6F162EdEDb761D0F33a2849,0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee,50000000000000000,3000000000000000
30986,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xb4EBD823f66e011D381bBad9249...,0x9b95045bda6a0dbba0f74cb7751c53e41909aaf21705...,15561481,0xeff8b47e3a9f1f0c11fb0d754be07b42a5ca7da9f666...,20,11,0xb4EBD823f66e011D381bBad924971C3850B8Ab68,2,0x740c178e10662bBb050BDE257bFA318dEfE3cabC,0x432f962D8209781da23fB37b6B59ee15dE7d9841,43000000000000000,1024000000000000000
30988,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xA96dC35B1250BCd2f72b6a89642...,0x798d295a6a34812fa419857a9a79dbcd54f07da095c9...,15561511,0xcbb593d023a670bbc236a4cd55745786c05d54be139f...,41,35,0xA96dC35B1250BCd2f72b6a896423A46D9130115b,2,0xe2ddf03ba8cDafd2Bb4884E52f7FB46df4FC7DC1,0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee,20000000000000000,280000000000000000


In [43]:
# save trade pairs with specifics to csv
query_pairs_trade.to_csv('notebook/data/pairs_trade.csv', index=False)